In [141]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from random import randint
import plotly.graph_objects as go
import plotly.express as px

os.chdir("/home/william/Desktop")

In [142]:
datosTicTacToe = pd.read_csv("tic-tac-toe.csv")

In [143]:
for x in datosTicTacToe:
    if x != " Gana-x":
        datosTicTacToe[x] = datosTicTacToe[x].astype('category')
datosTicTacToe.dtypes

cuadro-superior-izquierdo     category
 cuadro-superior-medio        category
 cuadro-superior-derecho      category
 cuadro-medio-izquierdo       category
 cuadro-medio-medio           category
  cuadro-medio-derecho        category
 cuadro-inferior-izquierdo    category
 cuadro-inferior-medio        category
 cuadro-inferior-derecho      category
 Gana-x                         object
dtype: object

In [144]:
datosTicTacToePosiblesResultados = list(set(datosTicTacToe.iloc[:, -1]))

In [145]:
class PrediccionBase:
    def __init__(self, datos):
        self.__datos = self.cleaning(datos)
        self.__precisionGlobal = 0
        self.__error_global = 0
        self.__verdaderosNegativos = 0
        self.__falsosPositivos = 0
        self.__falsosNegativos = 0
        self.__verdaderosPositivos = 0
        self.__reporte = 0
        self.__precision_category = 0

    @property
    def datos(self):
        return self.__datos

    @property
    def reporte(self):
        return self.__reporte

    def cleaning(self, datos):
        datos = datos.replace({'x': 0, 'o': 1, 'b': 2})
        return datos

    def entrenamiento(self):
        pass

    def generacionReporte(self, nombreDelModelo):
        dict = {
            "Modelo": [nombreDelModelo],
            "Precision Global": [self.__precisionGlobal],
            "Error Global": [self.__error_global],
            "Verdaderos Positivos": [self.__verdaderosPositivos],
            "Verdaderos Negativos": [self.__verdaderosNegativos],
            "Falsos Negativos": [self.__falsosNegativos],
            "Falsos Positivos": [self.__falsosPositivos]}
        self.__reporte = pd.DataFrame(dict).join(self.__precision_category)

    def analsis(self, MC, modelo):
        self.__verdaderosNegativos, self.__falsosPositivos, self.__falsosNegativos, self.__verdaderosPositivos = MC.ravel()
        self.__precisionGlobal = np.sum(MC.diagonal()) / np.sum(MC)
        self.__error_global = 1 - self.__precisionGlobal
        self.__precision_category = pd.DataFrame(MC.diagonal() / np.sum(MC, axis=1)).T
        self.__precision_category.columns = ["Precision Positiva (PP)", "Precision Negativa (PN)"]
        self.generacionReporte(modelo)
        return {"Matriz de Confusión": MC,
                "Precisión Global": self.__precisionGlobal,
                "Error Global": self.__error_global,
                "Precisión por categoría": self.__precision_category}

class PrediccionSVN(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia = None
        self.__instancia_potenciacion = None
        self.__x_train = []

    @property
    def instancia(self):
        return self.__instancia_potenciacion

    @property
    def x_test(self):
        return self.__x_test

    def obtenerVariablesImportantes(self):
        importancia = self.__instancia_potenciacion.feature_importances_
        print(importancia)
        etiquetas = self.__x_train.columns.values
        y_pos = np.arange(len(etiquetas))
        plt.figure(figsize=(10, 8))
        plt.barh(y_pos, importancia, align='center', alpha=0.5)
        plt.yticks(y_pos, etiquetas)

    def entrenamiento(self, kernel = "linear", entrenamiento="Suport Vector Machine",train_size=0.8):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]

        self.__instancia_potenciacion = SVC(kernel=kernel ,random_state=0)

        self.__x_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion.fit(self.__x_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entrenamiento)
        for k in indices:
             print("\n%s:\n%s" % (k, str(indices[k])))


class PrediccionKNeighbors(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia_potenciacion = None
        self.__x_test = None

    @property
    def instancia(self):
        return self.__instancia_potenciacion

    @property
    def x_test(self):
        return self.__x_test

    def entrenamiento(self, nucleo="auto", n_neighbors=3, train_size=0.80, entrenamiento="KNeighbors"):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]
        X_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion = KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=nucleo)
        self.__instancia_potenciacion.fit(X_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entrenamiento)
        # for k in indices:
        #     print("\n%s:\n%s" % (k, str(indices[k])))


class PrediccionADABoosting(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia = None
        self.__instancia_potenciacion = None
        self.__x_train = []

    @property
    def instancia(self):
        return self.__instancia_potenciacion

    @property
    def x_test(self):
        return self.__x_test

    def obtenerVariablesImportantes(self):
        importancia = self.__instancia_potenciacion.feature_importances_
        print(importancia)
        etiquetas = self.__x_train.columns.values
        y_pos = np.arange(len(etiquetas))
        plt.figure(figsize=(10, 8))
        plt.barh(y_pos, importancia, align='center', alpha=0.5)
        plt.yticks(y_pos, etiquetas)

    def entrenamiento(self, train_size=0.75, criterion="gini", splitter="best", min_samples_split=2,
                      entretenamiento="Bosques Aleatorios ADA Boosting"):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]
        self.__instancia = DecisionTreeClassifier(min_samples_split=min_samples_split, max_depth=None,
                                                  criterion=criterion, splitter=splitter)
        self.__instancia_potenciacion = AdaBoostClassifier(base_estimator=self.__instancia,
                                                           n_estimators=100, random_state=0)
        self.__x_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion.fit(self.__x_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entretenamiento)
        # for k in indices:
        #     print("\n%s:\n%s" % (k, str(indices[k])))


class PrediccionXGBoosting(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia = None
        self.__instancia_potenciacion = None
        self.__x_train = []

    @property
    def x_test(self):
        return self.__x_test
    @property
    def instancia(self):
        return self.__instancia_potenciacion

    def obtenerVariablesImportantes(self):
        importancia = self.__instancia_potenciacion.feature_importances_
        print(importancia)
        etiquetas = self.__x_train.columns.values
        y_pos = np.arange(len(etiquetas))
        plt.figure(figsize=(10, 8))
        plt.barh(y_pos, importancia, align='center', alpha=0.5)
        plt.yticks(y_pos, etiquetas)

    def entrenamiento(self, train_size=0.75, n_estimators=10, random_state=0, min_samples_split=2,
                      entrenamiento="Bosques Aleatorios XG Boosting"):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]

        self.__instancia_potenciacion = GradientBoostingClassifier(n_estimators=n_estimators, random_state=n_estimators,
                                                                   min_samples_split=min_samples_split, max_depth=None)

        self.__x_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion.fit(self.__x_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entrenamiento)
        # for k in indices:
        #     print("\n%s:\n%s" % (k, str(indices[k])))


class PrediccionRandomForest(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia = None
        self.__instancia_potenciacion = None
        self.__x_train = []

    @property
    def instancia(self):
        return self.__instancia_potenciacion

    @property
    def x_test(self):
        return self.__x_test

    def obtenerVariablesImportantes(self):
        importancia = self.__instancia_potenciacion.feature_importances_
        print(importancia)
        etiquetas = self.__x_train.columns.values
        y_pos = np.arange(len(etiquetas))
        plt.figure(figsize=(10, 8))
        plt.barh(y_pos, importancia, align='center', alpha=0.5)
        plt.yticks(y_pos, etiquetas)

    def entrenamiento(self, train_size=0.75, n_estimators=10, entrenamiento="Bosques Aleatorios"):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]

        self.__instancia_potenciacion = RandomForestClassifier(n_estimators=n_estimators, random_state=0)

        self.__x_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion.fit(self.__x_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entrenamiento)
        # for k in indices:
        #     print("\n%s:\n%s" % (k, str(indices[k])))


class PrediccionRandomForest(PrediccionBase):
    def __init__(self, datos):
        super().__init__(datos)
        self.__instancia = None
        self.__instancia_potenciacion = None
        self.__x_train = []

    @property
    def instancia(self):
        return self.__instancia_potenciacion

    @property
    def x_test(self):
        return self.__x_test

    def obtenerVariablesImportantes(self):
        importancia = self.__instancia_potenciacion.feature_importances_
        print(importancia)
        etiquetas = self.__x_train.columns.values
        y_pos = np.arange(len(etiquetas))
        plt.figure(figsize=(10, 8))
        plt.barh(y_pos, importancia, align='center', alpha=0.5)
        plt.yticks(y_pos, etiquetas)

    def entrenamiento(self, train_size=0.75, n_estimators=10, entrenamiento="Bosques Aleatorios"):
        x = self.datos.iloc[:, :-1]
        y = self.datos.iloc[:, -1]

        self.__instancia_potenciacion = RandomForestClassifier(n_estimators=n_estimators, random_state=0)

        self.__x_train, self.__x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0)
        self.__instancia_potenciacion.fit(self.__x_train, y_train)
        prediccion = self.__instancia_potenciacion.predict(self.__x_test)
        MC = confusion_matrix(y_test, prediccion)
        indices = self.analsis(MC, entrenamiento)
        # for k in indices:
        #     print("\n%s:\n%s" % (k, str(indices[k])))

Como podemos ver en el modelo predictivo de abajo, el modelo SVN RBF.
Tenemos una precision global del 0.875 lo cual es muy bueno, ademas, tiene un error global muy bajo.

La precision positiva es de un 67%, y una prediccion del 100% para la prediccion negativa.

In [146]:
#posibles Kernel : ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’
prediccionSVNTicTacRbf = PrediccionSVN(datos=datosTicTacToe)
prediccionSVNTicTacRbf.entrenamiento(kernel='rbf', entrenamiento="SVN RBF")



Matriz de Confusión:
[[ 50  24]
 [  0 118]]

Precisión Global:
0.875

Error Global:
0.125

Precisión por categoría:
   Precision Positiva (PP)  Precision Negativa (PN)
0                 0.675676                      1.0


En este otro caso podemos ver que el nucleo de sigmoide predice muy terrible, se puede ver en la precision global, y en que no esta prediciendo bien los verdaderos negativos

Tiene una precision alta, y un error global alto tambie, ademas las precisiones son muy malas tambien

In [147]:
prediccionSVNSigmoid = PrediccionSVN(datos=datosTicTacToe)
prediccionSVNSigmoid.entrenamiento(kernel='sigmoid',entrenamiento="SVN SIGMOID")


Matriz de Confusión:
[[ 2 72]
 [44 74]]

Precisión Global:
0.3958333333333333

Error Global:
0.6041666666666667

Precisión por categoría:
   Precision Positiva (PP)  Precision Negativa (PN)
0                 0.027027                 0.627119


In [148]:
prediccionADABosting = PrediccionADABoosting(datos=datosTicTacToe)
prediccionADABosting.entrenamiento(train_size=0.80)

In [149]:
prediccionXGBoosting = PrediccionXGBoosting(datos=datosTicTacToe)
prediccionXGBoosting.entrenamiento(train_size=0.80)

In [150]:
prediccionRandomForest = PrediccionRandomForest(datos=datosTicTacToe)
prediccionRandomForest.entrenamiento(train_size=0.80)

Como podemos ver en el dataset de abajo, rectifica lo que dijimos arriba, el modelo de RBF, de acuerdo a los datos predice mucho mejor que la otra.

In [151]:
pd.concat([prediccionSVNTicTacRbf.reporte, prediccionSVNSigmoid.reporte])

,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,SVN RBF,0.875000,0.125000,118,50,0,24,0.675676,1.000000
0,SVN SIGMOID,0.395833,0.604167,74,2,44,72,0.027027,0.627119


Basandonos con las tareas anteriores, podemos ver que los bosques aleatorios hacen una prediccion muy exacta, pero despues de los bosques aleatorios las support vector machine RBF, tiene unos resultados muy buenos


In [152]:
pd.concat([prediccionSVNTicTacRbf.reporte, prediccionSVNSigmoid.reporte, prediccionADABosting.reporte, prediccionXGBoosting.reporte,prediccionRandomForest.reporte])

,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,SVN RBF,0.875000,0.125000,118,50,0,24,0.675676,1.000000
0,SVN SIGMOID,0.395833,0.604167,74,2,44,72,0.027027,0.627119
0,Bosques Aleatorios ADA Boosting,0.890625,0.109375,115,56,3,18,0.756757,0.974576
0,Bosques Aleatorios XG Boosting,0.880208,0.119792,116,53,2,21,0.716216,0.983051
0,Bosques Aleatorios,0.901042,0.098958,112,61,6,13,0.824324,0.949153


In [153]:
datosWine = pd.read_csv("wine.csv")

El modelo linear predice excelente tiene una precision global del 98% y un error global del 0.10%, ademas si vemos las precisiones positivas podemos ver que tiene un 99% y la negativa un 96%
Este modelo es una maravilla para la prediccion!

In [154]:
prediccionSVNWineLinear = PrediccionSVN(datos=datosWine)
prediccionSVNWineLinear.entrenamiento(kernel='linear', entrenamiento="SVN Linear", train_size=0.75)
prediccionSVNWineLinear.reporte


Matriz de Confusión:
[[1224    3]
 [  14  384]]

Precisión Global:
0.9895384615384616

Error Global:
0.010461538461538411

Precisión por categoría:
   Precision Positiva (PP)  Precision Negativa (PN)
0                 0.997555                 0.964824


,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,SVN Linear,0.989538,0.010462,384,1224,14,3,0.997555,0.964824


Por otra parte el modelo SIGMOIDE esta fallando un poco mas que el modelo anterior, como podemos ver en la precision global, y ademas como podemos ver la matriz de correlacion esta hablando muy mal de la forma en la que predice este modelo

In [155]:
prediccionSVNWineSigmoid = PrediccionSVN(datos=datosWine)
prediccionSVNWineSigmoid.entrenamiento(kernel='sigmoid', entrenamiento="SVN SIGMOID", train_size=0.75)
prediccionSVNWineSigmoid.reporte


Matriz de Confusión:
[[1034  193]
 [ 214  184]]

Precisión Global:
0.7495384615384615

Error Global:
0.2504615384615385

Precisión por categoría:
   Precision Positiva (PP)  Precision Negativa (PN)
0                 0.842706                 0.462312


,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,SVN SIGMOID,0.749538,0.250462,184,1034,214,193,0.842706,0.462312


Reafimando lo anterior podemos ver que el modelo linear es mucho mucho mejor que el modelo sigmoide en este caso.

In [156]:
pd.concat([prediccionSVNWineLinear.reporte, prediccionSVNWineSigmoid.reporte])


,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,SVN Linear,0.989538,0.010462,384,1224,14,3,0.997555,0.964824
0,SVN SIGMOID,0.749538,0.250462,184,1034,214,193,0.842706,0.462312


En comparacion a tareas anteriores el Random Forest predice excelentemente, sin embargo podemos ver que el modelo de support vector machine linear tiene una muy buena posicion al igual que los metodos de ADA Boosting y XG Boosting

In [157]:
prediccionRandomForest = PrediccionRandomForest(datosWine)
prediccionRandomForest.entrenamiento(train_size=0.75)
prediccionRandomForest.reporte

prediccionADABosting = PrediccionADABoosting(datos=datosWine)
prediccionADABosting.entrenamiento(train_size=0.75)

prediccionXGBoosting = PrediccionXGBoosting(datos=datosWine)
prediccionXGBoosting.entrenamiento(train_size=0.75)

prediccionRandomForest = PrediccionRandomForest(datos=datosWine)
prediccionRandomForest.entrenamiento(train_size=0.75)

pd.concat([prediccionSVNWineLinear.reporte, prediccionSVNWineSigmoid.reporte, prediccionRandomForest.reporte, prediccionADABosting.reporte, prediccionXGBoosting.reporte])

,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,SVN Linear,0.989538,0.010462,384,1224,14,3,0.997555,0.964824
0,SVN SIGMOID,0.749538,0.250462,184,1034,214,193,0.842706,0.462312
0,Bosques Aleatorios,0.995692,0.004308,392,1226,6,1,0.999185,0.984925
0,Bosques Aleatorios ADA Boosting,0.985231,0.014769,384,1217,14,10,0.991850,0.964824
0,Bosques Aleatorios XG Boosting,0.986462,0.013538,384,1219,14,8,0.993480,0.964824


En comparacion con los data frame  de arriba, en el caso donde solo usamos las variables mas importantes, en especifico con el linear, empeora un poquito la precision global, las precisiones especificas

In [158]:
datosWine = pd.DataFrame(datosWine, columns=["volatil.acidez", "cloruros", "total.sulfuro.dioxido",
                                             "libre.sulfuro.dioxido", "sulfitos", "densidad", "tipo"])
prediccionSVNVariablesImportantes = PrediccionSVN(datosWine)
prediccionSVNVariablesImportantes.entrenamiento(kernel="linear",train_size=0.75)
prediccionSVNVariablesImportantes.reporte


Matriz de Confusión:
[[1211   16]
 [  23  375]]

Precisión Global:
0.976

Error Global:
0.02400000000000002

Precisión por categoría:
   Precision Positiva (PP)  Precision Negativa (PN)
0                  0.98696                 0.942211


,Modelo,Precision Global,Error Global,Verdaderos Positivos,Verdaderos Negativos,Falsos Negativos,Falsos Positivos,Precision Positiva (PP),Precision Negativa (PN)
0,Suport Vector Machine,0.976,0.024,375,1211,23,16,0.98696,0.942211


In [159]:
X = [1,1,1,3,1,3,1,3,1]
Y = [0,0,1,1,1,2,2,2,1]
Z = [1,2,2,4,3,3,1,1,0]
Clase = ["Rojo","Rojo","Rojo","Rojo","Rojo","Azul","Azul","Azul","Azul"]
dict = {"x": X, "y": Y, "z": Z, "Clase": Clase}
df = pd.DataFrame(dict)
fig = px.scatter_3d(df, x='x', y='y', z='z',
              color='Clase', color_discrete_sequence=["red", "blue"])

x, y = np.meshgrid(range(5), range(5))
hiperplano_rojo = x + y
hiperplano_azul = x + y - 2
hiperplano_optimo = x + y - 1

fig.add_trace(
    go.Surface(x=x, y=y, z=hiperplano_rojo, text='Hiperplano Rojo', showscale=False)
)

fig.add_trace(
    go.Surface(x=x, y=y, z=hiperplano_azul, text='Hiperplano Azul', showscale=False)
)

fig.add_trace(
    go.Surface(x=x, y=y, z=hiperplano_optimo, text='Hiperplano Optimo', showscale=False)
)


fig.show()

La distancia de cada margen es de 2 unidades